In [ ]:
import requests
from bs4 import BeautifulSoup
from math import ceil, nan
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    page = BeautifulSoup(req.text,"html.parser")
    return page

In [ ]:
url = "https://produits.migros.ch/poulet-crispy-14kg"
page = render_html(url)

In [ ]:
product = page.find("main",class_="section-top-padding")

print("-"*150)
print(product['data-track-view'])
print("-"*150)
print(product)

In [155]:
from pprint import pprint

scripts = page.find("script",{"type":"application/ld+json"})
scripts = json.loads(scripts.text)
for script in scripts:
    type_ = script['@type']
    print("-"*150)
    if type_ == "WebPage":
        categories = []
        for cat in script['breadcrumb']['itemListElement'][2:-1]:
            categories.append(cat['item']['name'])
        print("categories:",categories)
    else:
        product = {
            "name": script['name'],
            "image": script['image'],
            "brand": script['brand']['name']
        }
        print("product:",product)
print("-"*150)

------------------------------------------------------------------------------------------------------------------------------------------------------
categories: ['Supermarché', 'Denrées alimentaires', 'Produits surgelés', 'Viande', 'Volaille']
------------------------------------------------------------------------------------------------------------------------------------------------------
product: {'name': 'Poulet crispy 1.4kg', 'image': 'https://image.migros.ch/original/16deafbf80ee0a9a7688c38069b683dd337824a5/poulet-crispy-14kg.jpg', 'brand': 'Don Pollo'}
------------------------------------------------------------------------------------------------------------------------------------------------------


In [157]:
def find_details(page):

    def find_table(text, split=False):
        table = {}
        for row in text.find("table").find_all("tr"):
            td = row.find_all("td")
            key = td[0].get_text().strip().lower().replace(" ","_")
            if split:
                table.update({
                    key: td[1].get_text().strip().split(", ")
                })
            else:
                table.update({
                    key: td[1].get_text().strip()
                })
        return table

    def find_calories(detail):
        table = detail.find("table")
        for th in table.find("thead").find_all("tr")[1].find_all("th"):
            try:
                d = re.findall("\d+",th.get_text())
                if len(d) > 0:
                    netto = th.get_text().strip()
                    break
            except: pass
        for tag in table.find_all("td"):
            if "kcal" in tag.text:
                cal = tag.get_text().strip()
                cal = re.search("\d+ kcal|\d+kcal",cal)
                if cal != None:
                    cal = cal.group()
                    break
        return f"{cal} / {netto}"

    content = {}
    for div in page.find("div",class_="product-stage").find_next_siblings("div"):
        try:
            title = div.find("h4").get_text().strip().lower().replace(" ","_")
            detail = div.find("div",{"id":re.compile("my-panel-*"),"role":"tabpanel"})
            if title == "numéros_d'article":
                detail = find_table(detail,split=True)
            elif title == "valeurs_nutritionnelles":
                detail = find_calories(detail)
            else:
                detail = find_table(detail)
            content.update({title:detail})
        except: pass
    
    return content

find_details(page)

{'principales_caractéristiques': {'procédé_de_production': 'Préfrit, Pané',
  'emballage': 'Sac/sachet',
  'conseils_pour_la_préparation': 'Rôtir, Cuire au four, Peut être frit'},
 'développement_durable': {'production:': '96%',
  'transport:': '1%',
  'emballage:': '3%'},
 'ingrédients_et_allergènes': {'ingrédients': "viande de poulet 58% [origine: voir champ d'impression spontanée], chapelure (farine fleur de blé, levure, paprika, sel de cuisine), eau, huile de colza, préparation d'épices (amidon de blé [avec sel de cuisine], gluten de blé, farine de blé, stabilisant: diphosphates, extrait de levure [avec sel de cuisine], sucre, épices), panure (farine de blé, amidon de blé, sel de cuisine), sel de cuisine iodé.",
  'déclaration_des_allergènes': 'Peut contenir: céleri.'},
 'valeurs_nutritionnelles': '212 kcal / 100 g',
 'origine': {'marque_d’identification': 'CH 301',
  'mention_de_l’origine': "Élaboré en Suisse avec de la viande de poulet du Brésil/Thaïlande (l'origine exacte doit ê

In [ ]:
product_data = {
    "code" : "",
    "sources" : ["https://www.auchan.fr/auchan-saumon-fume-de-norvege-mini/pr-C1182526"],
    "image_url" : "",
    "small_image_url" : "",
    "product_name" : "",
    "brand" : "",
    "nutriscore" : "",
    "categories" : [""],
    "ingredients_text" : "",
    "calories" : "",
}